## 03- Entrenamiento del Modelo

En este notebook realizaremos los procedimientos necesarios para entrenar el modelo que permita establecer la probabilidad de que un cliente no renueve su producto con la compañía, basado en las diferentes dimensiones de información que se tengan del mismo al momento de emitir la póliza y que hemos explorado en los notebooks anteriores. Para ello usaremos las ventajas de la librería `scikit-learn`.

In [1]:
from model_parameters import *
import pandas as pd
import numpy as np

data=pd.read_csv("data_modelacion.csv",sep=";",dtype={'Asegurado__c':str,'NumeroPoliza__c':str,'CodigoTipoAsegurado__c':str,'churn':int,'ClaseVehiculo__c':str,'TipoVehiculo__c':str,'PuntoVenta__c':str,'MarcaVehiculo__c':str})

## Generar Particiones de Datos

Se debe generar una partición de la data para test(10%), el restante se usará para entrenamiento y validación.

In [2]:
#Definir esquema para valores faltantes
num_cols=[i for i in data.columns.values.tolist() if data[i].dtype!="O" and i not in ['churn','Profesion__pc','Asegurado__c','CodigoTipoAsegurado__c','PuntoVenta__c','RamoTecnico__c','Tipo_poliza_c']]
str_cols=[i for i in data.columns.values.tolist() if data[i].dtype not in ["int32","float","int64"] and i not in ['churn','NumeroPoliza__c','Producto__c','Asegurado__c','CodigoTipoAsegurado__c','PuntoVenta__c','RamoTecnico__c','Tipo_poliza_c']]
train,test=train_test_split(data, test_size=0.10,random_state=123)

## Modelos a Evaluar

Se procede a evaluar dos tipos de modelos de tipo boosting (XGboost,LightGBM), los cuales son ampliamente usados en el modelamiento predictivo. A su vez, se prueba un modelo sencillo como la regresión logística, para comparar el desempeño frente a modelos de mayor complejidad. El objetivo general de esta comparación es establecer qué tipo de modelo puede encontrar los aspectos más relevantes para establecer cuándo un cliente tiene mayor propensión a renovar su producto con la compañía.

In [3]:
from sklearn import set_config
set_config(display='diagram')

In [4]:
X_train,Y_train=train.drop(columns=["churn","Asegurado__c"]),train['churn'].values

In [5]:
## Pasos de imputación y estandarización
num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="constant",fill_value=0)), ("scaler", StandardScaler())]
)

def get_classifier_model(model_name:str,k_cv:int=5,scoring="f1") -> GridSearchCV:
    """retorna un objeto GridSearchCV para entrenar con @param model_name y k_cv-fold Cross Validation
    para clasificación binaria, la métrica de scoring de los modelos está dada por @param scoring
    """
    cat_transformer = OneHotEncoder(handle_unknown="error")
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", num_transformer, num_cols),
            ("cat", cat_transformer, str_cols),
        ],sparse_threshold=0
    )

    model=MODEL_LIST[model_name]
    pipe=Pipeline([
        ('cleaner',preprocessor),
        ('pca',PCA()),
        ('model',model)

    ], verbose=True)

    param_grid=MODEL_PARAMS[model_name]['param_grid']

    grid=GridSearchCV(pipe,param_grid,cv=k_cv,refit=True,scoring=scoring,verbose=True,n_jobs=-1)
    return grid

xgb=get_classifier_model("xgboost")
logistic=get_classifier_model("logistic_regression")
lightgbm=get_classifier_model("lightgbm")

In [6]:
xgb.fit(X_train,Y_train.ravel())

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing model, total=   5.5s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cleaner',
                                        ColumnTransformer(sparse_threshold=0,
                                                          transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=0,
                                                                                                        strategy='constant')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['n_prod_prev',
                                                                          'total_siniestros',
                                                                          'total_pagado_smmlv',
                                                                          'anios_ultimo_siniestro',
                                                                          'Activos__c',
                                                                          'AnnualRevenue',
                                                                          'MontoAnual__c',
                                                                          'Otros...
                                                                          'ClaseVehiculo__c',
                                                                          'TipoVehiculo__c',
                                                                          'EstadoCivil__pc',
                                                                          'Genero__pc'])])),
                                       ('pca', PCA()),
                                       ('model', GradientBoostingClassifier())],
                                verbose=True),
             n_jobs=-1,
             param_grid={'model__learning_rate': [0.001],
                         'model__max_depth': [5, 10, 20],
                         'model__min_samples_split': [2],
                         'model__n_estimators': [100],
                         'model__subsample': [0.9, 1],
                         'pca__n_components': [10, 20]},
             scoring='f1', verbose=True)

In [7]:
logistic.fit(X_train,Y_train.ravel())

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing model, total=  37.2s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cleaner',
                                        ColumnTransformer(sparse_threshold=0,
                                                          transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=0,
                                                                                                        strategy='constant')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['n_prod_prev',
                                                                          'total_siniestros',
                                                                          'total_pagado_smmlv',
                                                                          'anios_ultimo_siniestro',
                                                                          'Activos__c',
                                                                          'AnnualRevenue',
                                                                          'MontoAnual__c',
                                                                          'Otros...
                                                                          'tipo_prod_desc',
                                                                          'ClaseVehiculo__c',
                                                                          'TipoVehiculo__c',
                                                                          'EstadoCivil__pc',
                                                                          'Genero__pc'])])),
                                       ('pca', PCA()),
                                       ('model',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=4000,
                                                           solver='saga'))],
                                verbose=True),
             n_jobs=-1,
             param_grid={'model__l1_ratio': [0.1, 0.3, 0.5, 0.8, 1],
                         'model__penalty': ['elasticnet'],
                         'pca__n_components': [5, 10, 20]},
             scoring='f1', verbose=True)

In [15]:
lightgbm.fit(X_train,Y_train.ravel())

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.3s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cleaner',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=0,
                                                                                                        strategy='constant')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['n_prod_prev',
                                                                          'total_siniestros',
                                                                          'total_pagado_smmlv',
                                                                          'anios_ultimo_siniestro',
                                                                          'Activos__c',
                                                                          'AnnualRevenue',
                                                                          'MontoAnual__c',
                                                                          'OtrosIngresos__c',
                                                                          'Egres...
                                                                          'EstadoCivil__pc',
                                                                          'Genero__pc'])])),
                                       ('model', LGBMClassifier())],
                                verbose=True),
             param_grid={'model__colsample_bytree': [0.8],
                         'model__max_depth': [15],
                         'model__min_split_gain': [0.3],
                         'model__n_estimators': [100],
                         'model__num_leaves': [50], 'model__reg_alpha': [1.1],
                         'model__reg_lambda': [1.1], 'model__subsample': [0.9],
                         'model__subsample_freq': [20]},
             scoring='roc_auc', verbose=True)

## Verificar desempeño de los modelos

A continuación se proponen métricas generales para evaluar el desempeño de los modelos entrenados para las bases de entrenamiento y test.

In [15]:
confusion_matrix(Y_train.ravel(),logistic.predict(X_train))

array([[41271,  7371],
       [ 3153,  6168]], dtype=int64)

In [8]:
from utils import metrics

print("===Métricas para XGboost===\n")
print("\n\t Training\n")
_=metrics(Y_train.ravel(),xgb.predict(X_train))
print("\n\t Test \n")
_=metrics(test['churn'].values.ravel(),xgb.predict(test.drop(columns=["churn","Asegurado__c"])))
print("\n===Métricas para Regresión Logística===\n")
print("\n\t Training\n")
_=metrics(Y_train.ravel(),logistic.predict(X_train))
print("\n\t Test \n")
_=metrics(test['churn'].values.ravel(),logistic.predict(test.drop(columns=["churn","Asegurado__c"])))



===Métricas para XGboost===


	 Training

 Accuracy:0.8392 
 Precision: 0.0000 
 Recall: 0.0000 
 Kappa: 0.0000 
 F1-Score: 0.0000 

	 Test 

 Accuracy:0.8404 
 Precision: 0.0000 
 Recall: 0.0000 
 Kappa: 0.0000 
 F1-Score: 0.0000 

===Métricas para Regresión Logística===


	 Training

 Accuracy:0.8184 
 Precision: 0.4556 
 Recall: 0.6617 
 Kappa: 0.4313 
 F1-Score: 0.5396 

	 Test 

 Accuracy:0.8194 
 Precision: 0.4557 
 Recall: 0.6751 
 Kappa: 0.4368 
 F1-Score: 0.5441 


C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Exportar modelo para futuro uso

In [12]:
import pickle
import os

if not os.path.isdir("models"):
    os.mkdir("models")

with open("models/best_model.pickle","wb") as f:
    pickle.dump(logistic,f)